# Reductions and Local memory in numba-dpex

### Learning Objectives
- Use ND-range to show improvement in parallelism over the basic implementation.
- Use local memory to avoid repeated global memory access 
- Understand the usage of group barriers to synchronize all work-items and perform reduction
- Use atomic operation to perform reduction

## Reductions
A __reduction produces a single value by combining multiple values__ in an unspecified order, using an operator that is both associative and commutative (e.g. addition). Only the final value resulting from a reduction is of interest to the programmer.

A very common example is calculating __sum__ by adding a bunch of values. other examples are maximum and minumum

Parallelizing reductions can be tricky because of the nature of computation and accelerator hardware. Let's look at code examples showing how reduction can be performed on GPU using kernel invocation

In [ ]:
%%writefile lab/reduction_kernel.py
# SPDX-FileCopyrightText: 2020 - 2023 Intel Corporation
#
# SPDX-License-Identifier: Apache-2.0

import math

import dpctl
import numpy as np

import numba_dpex as ndpx


@ndpx.kernel
def sum_reduction_kernel(A, R, stride):
    i = ndpx.get_global_id(0)
    # sum two element
    R[i] = A[i] + A[i + stride]
    # store the sum to be used in nex iteration
    A[i] = R[i]


def sum_reduce(A):
    """Size of A should be power of two."""
    total = len(A)
    # max size will require half the size of A to store sum
    R = np.array(np.random.random(math.ceil(total / 2)), dtype=A.dtype)

    # Use the environment variable SYCL_DEVICE_FILTER to change the default device.
    # See https://github.com/intel/llvm/blob/sycl/sycl/doc/EnvironmentVariables.md#sycl_device_filter.
    device = dpctl.select_default_device()
    print("Using device ...")
    device.print_device_info()

    with dpctl.device_context(device):
        while total > 1:
            global_size = total // 2
            sum_reduction_kernel[ndpx.Range(global_size)](A, R, global_size)
            total = total // 2

    return R[0]


def test_sum_reduce():
    # This test will only work for size = power of two
    N = 2048
    #assert N % 2 == 0

    A = np.array(np.random.random(N), dtype=np.float32)
    A_copy = A.copy()

    actual = sum_reduce(A)
    expected = A_copy.sum()

    print("Actual:  ", actual)
    print("Expected:", expected)

    #assert expected - actual < 1e-2

    print("Done...")


if __name__ == "__main__":
    test_sum_reduce()
    

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_reduction_kernel.sh; if [ -x "$(command -v qsub)" ]; then ./q run_reduction_kernel.sh; else ./run_reduction_kernel.sh; fi


## ND-Range Kernels

Naive parallel kernel do not allow for performance optimizations at a hardware level. In these next two kernels we will utilize ND-Range kernels as a way to expresses parallelism enabling low level performance tuning by providing access to both global and local memory and mapping executions to compute units on hardware. The entire iteration space is divided into smaller groups called work-groups, work-items are organized into these work-groups and are scheduled on a single compute unit on the hardware.  Workgroup size must divide the entire ND-range size exactly in each dimension.  These sizes can all vary by hardware platform and by using the device queries below a developer can identify what is possible.  The workload must be considered to find the best mix of these values.

<img src="Assets/ndrange-subgroup.png">

The grouping of kernel executions into work-groups allows control of resource usage and load balance work distribution. The functionality of nd_range kernels is exposed via nd_range and nd_item classes. nd_range class represents a grouped execution range using global execution range and the local execution range of each work-group. nd_item class represents an individual instance of a kernel function and allows you to query for work-group range and index.



<img src="Assets/ndrange.png">

The work-group size depends on the accelerator hardware capability, so we set this size using command-line argument. Some hardware requre the matrix size to divide equally by the work-group size, we will use work-group size of 16x16 (256) by default which works for all the accelerator hardware we will be using to test, we will eventually use different work-group sizes to see how it impacts the performance.


## Shared Local Memory (SLM) Implementation

In a parallel algorithm, there is a high degree of reuse, so instead of loading values from global memory each time we can load the values into local memory and perform the computation.  This will reduce the latency of accessing the data values.  The difference between this implementation and the ND-range implementation is the reading is done from global memory in the case of ND-range each time and in this implementation they dat is loaded into local memory and then computed.  

When a work-group begins, the contents of its local memory are uninitialized, and local memory does not persist after a work-group finishes executing. Because of these properties, local memory may only be used for temporary storage while a work-group is executing.  For other devices though, such as many GPU devices, there are dedicated resources for local memory, and on these devices, communicating via local memory should perform better than communicating via global memory.

In SYCL’s memory model, local memory is a contiguous region of memory allocated per work group and is visible to all the work items in that group. Local memory is device-only and cannot be accessed from the host. From the perspective offers the device, the local memory is exposed as a contiguous array of a specific types. The maximum available local memory is hardware-specific. The SYCL local memory concept is analogous to CUDA’s shared memory concept.

Numba-dpex provides a special function numba_dpex.local.array to allocate local memory for a kernel. To simplify kernel development and accelerate communication between work-items in a work-group, SYCL defines a special local memory space specifically for communication between work-items in a work-group.

Local Address Space refers to memory objects that need to be allocated in local memory pool and are shared by all work-items of a work-group. Numba-dpex does not support passing arguments that are allocated in the local address space to @numba_dpex.kernel. Users are allowed to allocate static arrays in the local address space inside the @numba_dpex.kernel. In the example below numba_dpex.local.array(shape, dtype) is the API used to allocate a static array in the local address space:
These are used to compute an intermediate result which does not use global memory for repeated access for computation. 

Also notice that we used a barrier that helps to synchronize all of the work-items in the work-group. 

<img src="Assets/localmem.png">


### Group Barrier

When local accessor data is shared, work-group barriers are often required for work-item synchronization.

The `group_barrier` function synchronizes how each work-item views the state of memory. This type of synchronization operation is known as enforcing memory consistency or fencing memory. It ensures that the results of memory operations performed before the barrier are visible to other work-items after the
barrier.

A `group_barrier` is usually required right after a local accessor is modified by a work-item so that it is synchronized for all work-items before the local accessor can be accessed.

Below is an example of how to use local memory with barriers.

## Local Memory sample
The following DPC++ code shows a local-memory implementation of matrix multiplication: Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/local_memory_kernel.py
##==============================================================
## Copyright © Intel Corporation
##
## SPDX-License-Identifier: Apache-2.0
## =============================================================

import dpctl
import numpy as np
from numba import float32

import numba_dpex as dpex


def no_arg_barrier_support():
    """
    This example demonstrates the usage of numba_dpex's ``barrier``
    intrinsic function. The ``barrier`` function is usable only inside
    a ``kernel`` and is equivalent to OpenCL's ``barrier`` function.
    """

    @dpex.kernel
    def twice(A):
        i = dpex.get_global_id(0)
        d = A[i]
        # no argument defaults to global mem fence
        dpex.barrier()
        A[i] = d * 2

    N = 10
    arr = np.arange(N).astype(np.float32)
    print(arr)

    # Use the environment variable SYCL_DEVICE_FILTER to change the default device.
    # See https://github.com/intel/llvm/blob/sycl/sycl/doc/EnvironmentVariables.md#sycl_device_filter.
    device = dpctl.select_default_device()
    print("Using device ...")
    device.print_device_info()

    with dpctl.device_context(device):
        twice[N, dpex.DEFAULT_LOCAL_SIZE](arr)

    # the output should be `arr * 2, i.e. [0, 2, 4, 6, ...]`
    print(arr)


def local_memory():
    """
    This example demonstrates the usage of numba-dpex's `local.array`
    intrinsic function. The function is used to create a static array
    allocated on the devices local address space.
    """
    blocksize = 10

    @dpex.kernel
    def reverse_array(A):
        lm = dpex.local.array(shape=10, dtype=float32)
        i = dpex.get_global_id(0)

        # preload
        lm[i] = A[i]
        # barrier local or global will both work as we only have one work group
        dpex.barrier(dpex.LOCAL_MEM_FENCE)  # local mem fence
        # write
        A[i] += lm[blocksize - 1 - i]

    arr = np.arange(blocksize).astype(np.float32)
    print(arr)

    # Use the environment variable SYCL_DEVICE_FILTER to change the default device.
    # See https://github.com/intel/llvm/blob/sycl/sycl/doc/EnvironmentVariables.md#sycl_device_filter.
    device = dpctl.select_default_device()
    print("Using device ...")
    device.print_device_info()

    with dpctl.device_context(device):
        reverse_array[blocksize, dpex.DEFAULT_LOCAL_SIZE](arr)

    # the output should be `orig[::-1] + orig, i.e. [9, 9, 9, ...]``
    print(arr)


def main():
    no_arg_barrier_support()
    local_memory()

    print("Done...")


if __name__ == "__main__":
    main()
    

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_local_memory.sh; if [ -x "$(command -v qsub)" ]; then ./q run_local_memory.sh; else ./run_local_memory.sh; fi

## Reductions using Local memory and local barriers
One popular way of doing a reduction operation on GPUs is to create a number of work-groups and do a tree reduction in each work-group. In the kernel shown below, each work-item in the work-group participates in a reduction network to eventually sum up all the elements in that work-group.

All the intermediate results from the work-groups are then summed up by doing a serial reduction (if this intermediate set of results is large enough then we can do few more round(s) of tree reductions). This tree reduction algorithm takes advantage of the very fast synchronization operations among the work-items in a work-group. The performance of this kernel is highly dependent on the efficiency of the kernel launches, because a large number of kernels are launched

The following code shows a local-memory implementation of Reductions: Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/reduce_local_memory.py
# SPDX-FileCopyrightText: 2020 - 2023 Intel Corporation
#
# SPDX-License-Identifier: Apache-2.0

import dpctl
import dpctl.tensor as dpt
from numba import int32

import numba_dpex as ndpx


@ndpx.kernel
def sum_reduction_kernel(A, partial_sums):
    """
    The example demonstrates a reduction kernel implemented as a ``kernel``
    function.
    """
    local_id = ndpx.get_local_id(0)
    global_id = ndpx.get_global_id(0)
    group_size = ndpx.get_local_size(0)
    group_id = ndpx.get_group_id(0)

    local_sums = ndpx.local.array(64, int32)

    # Copy from global to local memory
    local_sums[local_id] = A[global_id]

    # Loop for computing local_sums : divide workgroup into 2 parts
    stride = group_size // 2
    while stride > 0:
        # Waiting for each 2x2 addition into given workgroup
        ndpx.barrier(ndpx.LOCAL_MEM_FENCE)

        # Add elements 2 by 2 between local_id and local_id + stride
        if local_id < stride:
            local_sums[local_id] += local_sums[local_id + stride]

        stride >>= 1

    if local_id == 0:
        partial_sums[group_id] = local_sums[0]


def sum_reduce(A):
    global_size = len(A)
    work_group_size = 64
    # nb_work_groups have to be even for this implementation
    nb_work_groups = global_size // work_group_size

    partial_sums = dpt.zeros(nb_work_groups, dtype=A.dtype, device=A.device)

    gs = ndpx.Range(global_size)
    ls = ndpx.Range(work_group_size)
    sum_reduction_kernel[ndpx.NdRange(gs, ls)](A, partial_sums)

    final_sum = 0
    # calculate the final sum in HOST
    for i in range(nb_work_groups):
        final_sum += int(partial_sums[i])

    return final_sum


def test_sum_reduce():
    N = 1024
    device = dpctl.select_default_device()
    A = dpt.ones(N, dtype=dpt.int32, device=device)

    print("Running Device + Host reduction")

    actual = sum_reduce(A)
    expected = N

    print("Actual:  ", actual)
    print("Expected:", expected)

    assert actual == expected

    print("Done...")


if __name__ == "__main__":
    test_sum_reduce()

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 reduce_local_memory.sh; if [ -x "$(command -v qsub)" ]; then ./q reduce_local_memory.sh; else ./reduce_local_memory.sh; fi

# Atomic Operations

Atomic operations enable __concurrent access to a memory location without introducing a data race__. When multiple atomic operations access the same memory, they are guaranteed not to overlap. 

To understand why atomic operations are necessary, let look at few kernel examples to perform reduction, addition of N number of elements:

#### Serial Computation with single_task
A simple way to perform reduction is by using a for-loop to add all items in a single_task kernel submission as show below, but it does not take advantage of parallelism in hardware.
```cpp
     for i in range(N):
        sum += data[i]
```

#### Parallel Computation with parallel_for may encounter race conditions
Using parallel_for for kernel submission will enable multiple work-items to execute concurrently but multiple work-item may try to update the same output variable causing __race conditions__.

Parallel Computation with atomic operation avoid race conditions when multiple work-items are trying to update the same memory location

The following code shows Atomics implementation of Reductions: Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.


In [ ]:
%%writefile lab/atomics_kernel.py
import dpnp as np

import numba_dpex as ndpx


@ndpx.kernel
def atomic_reduction(a):
    idx = ndpx.get_global_id(0)
    ndpx.atomic.add(a, 0, a[idx])


def main():
    N = 1024
    a = np.arange(N)

    print("Using device ...")
    print(a.device)

    atomic_reduction[ndpx.Range(N)](a)
    print("Reduction sum =", a[0])

    print("Done...")


if __name__ == "__main__":
    main()

In [ ]:
! chmod 755 q; chmod 755 run_atomics.sh; if [ -x "$(command -v qsub)" ]; then ./q run_atomics.sh; else ./run_atomics.sh; fi

### Recursive Reduction

There are multiple ways of implementing reduction using numba_dpex. Here we demonstrate another way of implementing reduction using recursion to compute partial reductions in separate kernels.

The following code shows a local-memory implementation of Recursive Reductions: Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.


In [ ]:
%%writefile lab/recursive_reduction_kernel.py
# SPDX-FileCopyrightText: 2020 - 2023 Intel Corporation
#
# SPDX-License-Identifier: Apache-2.0

"""
There are multiple ways of implementing reduction using numba_ndpx. Here we
demonstrate another way of implementing reduction using recursion to compute
partial reductions in separate kernels.
"""

import dpctl
import dpctl.tensor as dpt
from numba import int32

import numba_dpex as ndpx


@ndpx.kernel
def sum_reduction_kernel(A, input_size, partial_sums):
    local_id = ndpx.get_local_id(0)
    global_id = ndpx.get_global_id(0)
    group_size = ndpx.get_local_size(0)
    group_id = ndpx.get_group_id(0)

    local_sums = ndpx.local.array(64, int32)

    local_sums[local_id] = 0

    if global_id < input_size:
        local_sums[local_id] = A[global_id]

    # Loop for computing local_sums : divide workgroup into 2 parts
    stride = group_size // 2
    while stride > 0:
        # Waiting for each 2x2 addition into given workgroup
        ndpx.barrier(ndpx.LOCAL_MEM_FENCE)

        # Add elements 2 by 2 between local_id and local_id + stride
        if local_id < stride:
            local_sums[local_id] += local_sums[local_id + stride]

        stride >>= 1

    if local_id == 0:
        partial_sums[group_id] = local_sums[0]


def sum_recursive_reduction(size, group_size, Dinp, Dpartial_sums):
    result = 0
    nb_work_groups = 0
    passed_size = size

    if size <= group_size:
        nb_work_groups = 1
    else:
        nb_work_groups = size // group_size
        if size % group_size != 0:
            nb_work_groups += 1
            passed_size = nb_work_groups * group_size

    gr = ndpx.Range(passed_size)
    lr = ndpx.Range(group_size)

    sum_reduction_kernel[ndpx.NdRange(gr, lr)](Dinp, size, Dpartial_sums)

    if nb_work_groups <= group_size:
        sum_reduction_kernel[ndpx.NdRange(lr, lr)](
            Dpartial_sums, nb_work_groups, Dinp
        )
        result = int(Dinp[0])
    else:
        result = sum_recursive_reduction(
            nb_work_groups, group_size, Dpartial_sums, Dinp
        )

    return result


def sum_reduce(A):
    global_size = len(A)
    work_group_size = 64
    nb_work_groups = global_size // work_group_size
    if (global_size % work_group_size) != 0:
        nb_work_groups += 1

    partial_sums = dpt.zeros(nb_work_groups, dtype=A.dtype, device=A.device)
    result = sum_recursive_reduction(
        global_size, work_group_size, A, partial_sums
    )

    return result


def test_sum_reduce():
    N = 20000
    device = dpctl.select_default_device()
    A = dpt.ones(N, dtype=dpt.int32, device=device)

    print("Running recursive reduction")

    actual = sum_reduce(A)
    expected = N

    print("Actual:  ", actual)
    print("Expected:", expected)

    assert actual == expected

    print("Done...")


if __name__ == "__main__":
    test_sum_reduce()


In [ ]:
! chmod 755 q; chmod 755 run_recursive_reduction.sh; if [ -x "$(command -v qsub)" ]; then ./q run_recursive_reduction.sh; else ./run_recursive_reduction.sh; fi

### Private Address Space
Private Address Space refers to memory objects that are local to each work-item and is not shared with any other work-item. In the example below numba_dpex.private.array(shape, dtype) is the API used to allocate a static array in the private address space:

<img src="Assets/workgroup.png">

## Private Memory sample

The following code shows a Private-memory implementation of numba-dpex: Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/private_memory_kernel.py
# SPDX-FileCopyrightText: 2020 - 2023 Intel Corporation
#
# SPDX-License-Identifier: Apache-2.0

import dpctl
import dpctl.tensor as dpt
import numpy as np
from numba import float32

import numba_dpex as ndpx


def private_memory():
    """
    This example demonstrates the usage of numba_dpex's `private.array`
    intrinsic function. The function is used to create a static array
    allocated on the devices private address space.
    """

    @ndpx.kernel
    def private_memory_kernel(A):
        memory = ndpx.private.array(shape=1, dtype=np.float32)
        i = ndpx.get_global_id(0)

        # preload
        memory[0] = i
        ndpx.barrier(ndpx.LOCAL_MEM_FENCE)  # local mem fence

        # memory will not hold correct deterministic result if it is not
        # private to each thread.
        A[i] = memory[0] * 2

    N = 4
    device = dpctl.select_default_device()

    arr = dpt.zeros(N, dtype=dpt.float32, device=device)
    orig = np.arange(N).astype(np.float32)

    print("Using device ...")
    device.print_device_info()

    global_range = ndpx.Range(N)
    local_range = ndpx.Range(N)
    private_memory_kernel[ndpx.NdRange(global_range, local_range)](arr)

    arr_out = dpt.asnumpy(arr)
    np.testing.assert_allclose(orig * 2, arr_out)
    # the output should be `orig[i] * 2, i.e. [0, 2, 4, ..]``
    print(arr_out)


def main():
    private_memory()

    print("Done...")


if __name__ == "__main__":
    main()
    

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_private_memory.sh; if [ -x "$(command -v qsub)" ]; then ./q run_private_memory.sh; else ./run_private_memory.sh; fi

## Summary
In this module you will have learned the following:
* Implementation of reduction operation using Local memory in numba-dpex
* Implementation of reduction operation using atomics in numba-dpex
* Implementation of Private memory in numba-dpex